### Verarbeitung der Daten: Deutschlandatlas

In [1]:
import pandas as pd
from pathlib import Path
from collections import Counter

# ============================================================
# Projektbasis ermitteln (funktioniert im .py und halbwegs im Notebook)
# ============================================================
try:
    # wenn als .py ausgeführt
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    # wenn im Notebook ausgeführt
    BASE_DIR = Path.cwd()

# jetzt alles relativ zu BASE_DIR
DATA_INPUT_DIR = BASE_DIR / "data_input"

# Pfad zur großen Ursprungsdatei (relativ)
SRC_PATH = DATA_INPUT_DIR / "Deutschlandatlas.xlsx"

# Ausgabeordner (relativ)
OUTPUT_DIR = DATA_INPUT_DIR / "deutschlandatlas_processed"
OUTPUT_DIR.mkdir(exist_ok=True)
# ============================================================

# Präfixe + gewünschte Namensspalte pro Datei
PREFIX_CONFIG = {
    "Deutschlandatlas_GEM": "Gemeindename",
    "Deutschlandatlas_KRS": "Kreisname",
    "Deutschlandatlas_VBGEM": "Gemeindeverbandsname",
}


def normalize_gkz(val, width: int = 8) -> str:
    if pd.isna(val):
        return ""
    s = str(val).strip()
    try:
        num = int(float(s.replace(" ", "")))
        return str(num).zfill(width)
    except (ValueError, TypeError):
        if "." in s:
            s = s.split(".")[0]
        s = s.replace(" ", "")
        return s.zfill(width)


def longest_name_per_gkz(df_names: pd.DataFrame, name_col: str) -> pd.DataFrame:
    df = df_names.copy()
    df["__len__"] = df[name_col].fillna("").astype(str).str.len()
    df = (
        df.sort_values(["GKZ", "__len__"], ascending=[True, False])
          .drop_duplicates(subset=["GKZ"], keep="first")
          .drop(columns=["__len__"])
    )
    return df


def build_vbgem_gkz_canonical_map(all_names: pd.DataFrame, name_col: str) -> dict:
    fix_map = {}
    for name, grp in all_names.groupby(name_col):
        gkz_list = [g for g in grp["GKZ"].dropna().unique()]
        if not gkz_list:
            continue
        zero_gkz = [g for g in gkz_list if g.startswith("0")]
        if zero_gkz:
            canonical = min(zero_gkz)
        else:
            canonical = min(gkz_list)
        for g in gkz_list:
            fix_map[g] = canonical
    return fix_map


def main():
    # komplette Excel einlesen
    xls = pd.ExcelFile(SRC_PATH)

    # Sheets nach Präfix sammeln
    grouped = {p: [] for p in PREFIX_CONFIG.keys()}
    for sheet in xls.sheet_names:
        for prefix in PREFIX_CONFIG.keys():
            if sheet.startswith(prefix):
                df = pd.read_excel(xls, sheet_name=sheet)
                grouped[prefix].append((sheet, df))
                break

    # jetzt je Präfix abarbeiten
    for prefix, sheets in grouped.items():
        if not sheets:
            continue

        target_name_col = PREFIX_CONFIG[prefix]

        name_frames = []
        dataframes_for_merge = []

        for sheet_name, df in sheets:
            if df.shape[1] < 1:
                continue

            # erste Spalte = GKZ
            gkz_col = df.columns[0]
            df[gkz_col] = df[gkz_col].apply(normalize_gkz)

            # zweite Spalte = Name einsammeln
            if df.shape[1] >= 2:
                current_name_col = df.columns[1]
                tmp = df[[gkz_col, current_name_col]].copy()
                tmp = tmp.rename(columns={gkz_col: "GKZ", current_name_col: target_name_col})
                name_frames.append(tmp)

            # GKZ vereinheitlichen
            df = df.rename(columns={gkz_col: "GKZ"})

            # Namen aus diesem Sheet entfernen – später sauber wieder mergen
            if df.shape[1] >= 2:
                second_col = df.columns[1]
                if second_col != "GKZ":
                    df = df.drop(columns=[second_col], errors="ignore")

            dataframes_for_merge.append(df)

        # VBGEM-Sonderfall
        if prefix == "Deutschlandatlas_VBGEM" and name_frames:
            all_names_raw = pd.concat(name_frames, ignore_index=True)
            fix_map = build_vbgem_gkz_canonical_map(all_names_raw, target_name_col)

            fixed_name_frames = []
            for nf in name_frames:
                nf = nf.copy()
                nf["GKZ"] = nf["GKZ"].map(fix_map).fillna(nf["GKZ"])
                fixed_name_frames.append(nf)
            name_frames = fixed_name_frames

            fixed_dataframes_for_merge = []
            for df in dataframes_for_merge:
                df = df.copy()
                df["GKZ"] = df["GKZ"].map(fix_map).fillna(df["GKZ"])
                fixed_dataframes_for_merge.append(df)
            dataframes_for_merge = fixed_dataframes_for_merge

        # doppelte Spaltennamen behandeln
        level = prefix.rsplit("_", 1)[-1]  # "GEM", "KRS", "VBGEM"

        all_cols = []
        for df in dataframes_for_merge:
            all_cols.extend([c for c in df.columns if c != "GKZ"])

        counts = Counter(all_cols)
        duplicate_cols = {col for col, cnt in counts.items() if cnt > 1}

        if duplicate_cols:
            fixed_dfs = []
            for df in dataframes_for_merge:
                rename_dict = {c: f"{level}_{c}" for c in df.columns if c in duplicate_cols}
                df = df.rename(columns=rename_dict)
                fixed_dfs.append(df)
            dataframes_for_merge = fixed_dfs

        # Namen bereinigen
        if name_frames:
            all_names = pd.concat(name_frames, ignore_index=True)
            all_names = all_names[all_names[target_name_col].notna() & (all_names[target_name_col] != "")]
            unique_names = longest_name_per_gkz(all_names, target_name_col)
        else:
            unique_names = pd.DataFrame(columns=["GKZ", target_name_col])

        # alle Sheets über GKZ mergen
        merged = None
        for df in dataframes_for_merge:
            if merged is None:
                merged = df
            else:
                merged = pd.merge(merged, df, on="GKZ", how="outer")

        # Namen zurückmergen
        final_df = pd.merge(unique_names, merged, on="GKZ", how="right")

        # GKZ säubern + sortieren
        final_df["GKZ"] = final_df["GKZ"].apply(lambda x: normalize_gkz(x, 8)).astype(str)
        final_df = final_df.sort_values(["GKZ"])

        # relativ speichern
        out_parquet = OUTPUT_DIR / f"{prefix}_merged.parquet"
        final_df.to_parquet(out_parquet, index=False)
        print(f"[OK] geschrieben: {out_parquet}")

    print("Fertig. Dateien liegen in:", OUTPUT_DIR)


if __name__ == "__main__":
    main()

[OK] geschrieben: c:\Users\marius.brede\Nextcloud\tmp\dev\Zeitreihenanalyse - Dash Dashboard\Version 19\geospacial\data_input\deutschlandatlas_processed\Deutschlandatlas_GEM_merged.parquet
[OK] geschrieben: c:\Users\marius.brede\Nextcloud\tmp\dev\Zeitreihenanalyse - Dash Dashboard\Version 19\geospacial\data_input\deutschlandatlas_processed\Deutschlandatlas_KRS_merged.parquet
[OK] geschrieben: c:\Users\marius.brede\Nextcloud\tmp\dev\Zeitreihenanalyse - Dash Dashboard\Version 19\geospacial\data_input\deutschlandatlas_processed\Deutschlandatlas_VBGEM_merged.parquet
Fertig. Dateien liegen in: c:\Users\marius.brede\Nextcloud\tmp\dev\Zeitreihenanalyse - Dash Dashboard\Version 19\geospacial\data_input\deutschlandatlas_processed


### Erstellung der Maps

In [21]:
import json
from pathlib import Path

import pandas as pd
import folium
import branca.colormap as cm

# ============================================================
# Basis ermitteln (Notebook oder .py)
# ============================================================
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()

DATA_INPUT_DIR = BASE_DIR / "data_input"

# ============================================================
# BASIS-PFADE (relativ)
# ============================================================
# GeoJSON / Shapes
BASE_KRS = DATA_INPUT_DIR / "Shapefiles - KRS"
BASE_GEM = DATA_INPUT_DIR / "Shapefiles - GEM"
BASE_VBGEM = DATA_INPUT_DIR / "Shapefiles - VBGEM"

# parquet-Dateien aus dem Aufbereitungsscript
PROCESSED_DIR = DATA_INPUT_DIR / "deutschlandatlas_processed"

# Label / Meta
INDICATOR_CSV_PATH = DATA_INPUT_DIR / "deutschlandatlas_services_mit_counts.csv"
INDICATOR_META_PATH = DATA_INPUT_DIR / "deutschlandatlas_indikatoren.csv"

# Output (relativ)
OUTPUT_MAP_DIR = BASE_DIR / "data_output"
OUTPUT_MAP_DIR.mkdir(parents=True, exist_ok=True)

DEBUG = True
FAST_MODE = True

# ============================================================
# Konstanten
# ============================================================
MISSING_VALUES = frozenset({
    None,
    "",
    "nan",
    "-9999", "-9999.0", "-9999,0",
    "-99999", "-99999.0", "-99999,0",
    "-999999", "-999999.0", "-999999,0",
})
SUFFIX_DROP = frozenset({"ha2023", "ha2022", "ha2021", "ha2020", "za2023", "za2022", "3857"})
INDICATOR_PREFIXES = frozenset({"p", "v", "pendel", "teilz", "bev", "beschq", "kbetr"})
KEY_COLUMN_CANDIDATES = ("GKZ", "gkz", "Gebietskennziffer", "gebietskennziffer", "KRS", "AGS", "ags", "VBG", "vbg")
GKZ_PROPERTIES = ("GKZ", "Gebietskennziffer", "VBG")
NAME_FIELD_CANDIDATES = ("Gemeindename", "Gemeindeverbandsname", "Kreisname", "NAME", "GEN")


def log(*args):
    if DEBUG:
        print(*args)


def normalize_gkz(val, width=8) -> str:
    if val is None:
        return ""
    s = str(val).strip()
    if not s:
        return ""
    try:
        return str(int(float(s.replace(" ", "")))).zfill(width)
    except (ValueError, AttributeError):
        return s.split(".")[0].replace(" ", "").zfill(width)


def is_missing(val) -> bool:
    if val is None:
        return True
    s = str(val).strip().lower()
    return s in MISSING_VALUES


def detect_key_col(columns):
    for candidate in KEY_COLUMN_CANDIDATES:
        if candidate in columns:
            return candidate
    return columns[0]


def _strip_suffixes(parts: list[str]) -> list[str]:
    while parts and parts[-1] in SUFFIX_DROP:
        parts.pop()
    return parts


def canonicalize_indicator_name(name: str) -> str:
    if not name:
        return ""
    parts = str(name).strip().lower().split("_")
    parts = _strip_suffixes(parts)
    if not parts:
        return ""
    if len(parts) >= 2 and parts[0] in INDICATOR_PREFIXES:
        return f"{parts[0]}_{parts[1]}"
    return "_".join(parts)


def extract_long_from_layers(layers_val: str) -> str | None:
    if not isinstance(layers_val, str) or not layers_val:
        return None
    txt = layers_val.strip()
    if ": " in txt:
        txt = txt.split(": ", 1)[1]
    if " (Features:" in txt:
        txt = txt.split(" (Features:", 1)[0]
    return txt.strip() or None


def load_indicator_label_map(csv_path: Path) -> dict[str, str]:
    log(f"[labels] Lade Indikator-Labels aus {csv_path}")
    if not csv_path.exists():
        log("[labels] Datei existiert nicht – Label-Mapping wird leer.")
        return {}

    try:
        df = pd.read_csv(csv_path, dtype=str)
    except Exception as e:
        log(f"[labels] CSV konnte nicht gelesen werden: {e}")
        return {}

    cols_lower = {c.lower(): c for c in df.columns}
    title_col = cols_lower.get("title")
    layers_col = cols_lower.get("layers")
    tf_col = cols_lower.get("totalfeatures")

    if not title_col:
        log("[labels] Keine 'title'-Spalte gefunden – Mapping bleibt leer.")
        return {}

    mapping: dict[str, dict] = {}
    for _, row in df.iterrows():
        short_raw = str(row.get(title_col, "")).strip()
        if not short_raw:
            continue

        canon = canonicalize_indicator_name(short_raw)
        long_name = extract_long_from_layers(row.get(layers_col, "")) if layers_col else None
        long_name = long_name or short_raw

        score = 0
        if tf_col:
            try:
                score = int(float(str(row.get(tf_col, "0")).replace(",", ".")))
            except (ValueError, AttributeError):
                pass

        if canon not in mapping or score > mapping[canon]["score"]:
            mapping[canon] = {"long": long_name, "score": score}

    final_map = {k: v["long"] for k, v in mapping.items()}
    log(f"[labels] Fertig, {len(final_map)} Label-Einträge.")
    return final_map


def load_indicator_meta(csv_path: Path) -> dict[str, dict]:
    log(f"[meta] Lade Indikator-Metadaten aus {csv_path}")
    if not csv_path.exists():
        log("[meta] Datei existiert nicht – Metadaten bleiben leer.")
        return {}

    df = pd.read_csv(csv_path, dtype=str)
    cols_lower = {c.lower(): c for c in df.columns}

    code_col = cols_lower.get("code") or cols_lower.get("indikator_code") or cols_lower.get("indicator_code") or list(df.columns)[0]
    name_col = cols_lower.get("name") or cols_lower.get("indikator_name") or cols_lower.get("langname") or None
    desc_col = cols_lower.get("beschreibung") or cols_lower.get("description") or None

    meta: dict[str, dict] = {}
    for _, row in df.iterrows():
        raw_code = str(row.get(code_col, "")).strip()
        if not raw_code:
            continue
        canon = canonicalize_indicator_name(raw_code)
        meta_entry = {
            "name": str(row.get(name_col, "")).strip() if name_col else "",
            "desc": str(row.get(desc_col, "")).strip() if desc_col else "",
            "raw": raw_code,
        }
        meta[canon] = meta_entry

    log(f"[meta] Fertig, {len(meta)} Metadaten-Einträge.")
    return meta


def build_map(geojson_data: dict, value_field: str, display_name: str | None = None):
    feats = geojson_data.get("features", [])
    if not feats:
        log("[WARNUNG] Keine Features zum Visualisieren vorhanden.")
        return

    m = folium.Map(location=[51.163, 10.447], zoom_start=6, tiles="cartodbpositron")

    base_props = feats[0]["properties"]
    tooltip_fields = []
    for cand in ("GKZ", "Gemeindename", "Gemeindeverbandsname", "Kreisname", "indicator_name", value_field):
        if cand in base_props:
            tooltip_fields.append(cand)

    cleaned_vals = []
    for f in feats:
        raw_v = f["properties"].get(value_field)
        if is_missing(raw_v):
            continue
        try:
            cleaned_vals.append(float(str(raw_v).replace(",", ".")))
        except (ValueError, TypeError):
            continue

    vmin, vmax = (min(cleaned_vals), max(cleaned_vals)) if cleaned_vals else (0, 1)

    legend_caption = display_name or value_field
    colormap = cm.linear.YlOrRd_09.scale(vmin, vmax)
    colormap.caption = legend_caption

    def style_function(feature):
        v = feature["properties"].get(value_field)
        if is_missing(v):
            return {
                "fillColor": "#cccccc",
                "color": "black",
                "weight": 0.3,
                "fillOpacity": 0.7,
            }
        try:
            vv = float(str(v).replace(",", "."))
            color = colormap(vv)
        except (ValueError, TypeError):
            color = "#cccccc"
        return {
            "fillColor": color,
            "color": "black",
            "weight": 0.3,
            "fillOpacity": 0.7,
        }

    folium.GeoJson(
        geojson_data,
        name=legend_caption,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=tooltip_fields) if tooltip_fields else None,
    ).add_to(m)

    colormap.add_to(m)
    folium.LayerControl().add_to(m)

    out_html = OUTPUT_MAP_DIR / "output_map.html"
    m.save(str(out_html))
    log(f"[OK] HTML gespeichert: {out_html}")


def process_one(base_dir: Path,
                prefix: str,
                indicator_labels: dict[str, str],
                indicator_meta: dict[str, dict],
                parquet_path: Path):
    geojson_path = base_dir / f"{prefix}_Map.geojson"

    with open(geojson_path, "r", encoding="utf-8") as f:
        gj = json.load(f)

    # GKZ im GeoJSON normalisieren
    for feat in gj.get("features", []):
        props = feat.get("properties", {})
        for gkz_prop in GKZ_PROPERTIES:
            if gkz_prop in props:
                props["GKZ"] = normalize_gkz(props[gkz_prop])
                break

    # parquet laden
    df = pd.read_parquet(parquet_path)
    df = df.astype(str).replace({"nan": None})

    key_col = detect_key_col(df.columns)
    print(f"→ [{prefix}] erkannte Schlüsselspalte: {key_col}")

    df["GKZ"] = df[key_col].apply(normalize_gkz)

    # Raw-Index aus Meta
    meta_by_raw = {v["raw"].lower(): v for v in indicator_meta.values() if v.get("raw")}

    print(f"\nVerfügbare Spalten in {parquet_path.name}:")
    for i, c in enumerate(df.columns, 1):
        if c in ("GKZ", "Gemeindename", "Kreisname", "Gemeindeverbandsname"):
            print(f"  {i}: {c}")
            continue

        meta_entry = meta_by_raw.get(c.lower())
        if not meta_entry:
            canon = canonicalize_indicator_name(c)
            meta_entry = indicator_meta.get(canon)

        if meta_entry and meta_entry.get("name"):
            long_label_for_display = meta_entry["name"]
        else:
            canon = canonicalize_indicator_name(c)
            long_label_for_display = indicator_labels.get(canon, "")

        if long_label_for_display:
            print(f"  {i}: {c} – {long_label_for_display}")
        else:
            print(f"  {i}: {c}")

    user_in = input(f"\n[{prefix}] Spalte wählen (Nummer oder Name): ").strip()
    value_col = df.columns[int(user_in) - 1] if user_in.isdigit() else user_in
    print(f"→ [{prefix}] nehme Spalte: {value_col}")

    # Meta für gewählten Indikator
    meta_entry = meta_by_raw.get(value_col.lower())
    if not meta_entry:
        canon = canonicalize_indicator_name(value_col)
        meta_entry = indicator_meta.get(canon, {})

    canon = canonicalize_indicator_name(value_col)
    long_label = indicator_labels.get(canon)
    if meta_entry.get("name"):
        long_label = meta_entry["name"]
    indicator_desc = meta_entry.get("desc", "")

    if long_label:
        log(f"[labels/meta] Mapping gefunden: '{value_col}' -> '{long_label}'")
    else:
        long_label = value_col

    val_map = dict(zip(df["GKZ"], df[value_col]))
    name_field = next((cand for cand in NAME_FIELD_CANDIDATES if cand in df.columns), None)

    merged = 0
    tab_rows = []

    for feat in gj.get("features", []):
        props = feat.get("properties", {})
        gkz = props.get("GKZ")
        if not gkz:
            continue

        v = val_map.get(gkz)
        row = {"GKZ": gkz, "indicator_code": value_col, "indicator_name": long_label}

        if name_field:
            name_vals = df.loc[df["GKZ"] == gkz, name_field]
            if not name_vals.empty:
                row["name"] = name_vals.iloc[0]
            else:
                row["name"] = props.get(name_field, "")
        else:
            row["name"] = props.get("Gemeindename") or props.get("Kreisname") or props.get("Gemeindeverbandsname") or ""

        if indicator_desc:
            row["indicator_desc"] = indicator_desc

        if v is None or is_missing(v):
            props["indicator_code"] = value_col
            props["indicator_name"] = long_label
            if indicator_desc:
                props["indicator_desc"] = indicator_desc

            row[value_col] = None
            tab_rows.append(row)
            continue

        keep = {"GKZ": gkz, value_col: v, "indicator_code": value_col, "indicator_name": long_label}
        if indicator_desc:
            keep["indicator_desc"] = indicator_desc

        if name_field:
            name_vals = df.loc[df["GKZ"] == gkz, name_field]
            if not name_vals.empty:
                keep[name_field] = name_vals.iloc[0]

        feat["properties"] = keep
        merged += 1

        row[value_col] = v
        tab_rows.append(row)

    print(f"[{prefix}] {merged} Features gemerged.")

    # GeoJSON immer in Output
    out_geo = OUTPUT_MAP_DIR / "output_map.geojson"
    with open(out_geo, "w", encoding="utf-8") as f:
        json.dump(gj, f, ensure_ascii=False, separators=(",", ":"))
    print(f"[{prefix}] GeoJSON geschrieben: {out_geo}")

    # HTML immer in Output
    build_map(gj, value_col, display_name=long_label)

    # Tabellen-CSV immer in Output
    out_table = OUTPUT_MAP_DIR / "output_map_data.csv"
    df_out = pd.DataFrame(tab_rows)
    cols_order = ["GKZ", "name", value_col, "indicator_code", "indicator_name", "indicator_desc"]
    df_out = df_out.reindex(columns=cols_order)
    df_out.to_csv(out_table, index=False, encoding="utf-8")
    print(f"[{prefix}] Tabellen-CSV geschrieben: {out_table}")


def main():
    indicator_labels = load_indicator_label_map(INDICATOR_CSV_PATH)
    indicator_meta = load_indicator_meta(INDICATOR_META_PATH)

    print("\nWelche Ebene willst du mappen?")
    print("  1 = Kreise (KRS)")
    print("  2 = Gemeinden (GEM)")
    print("  3 = Gemeindeverband (VBGEM)")

    choice = input("\nAuswahl: ").strip()

    mapping = {
        "1": (BASE_KRS, "KRS", PROCESSED_DIR / "Deutschlandatlas_KRS_merged.parquet"),
        "2": (BASE_GEM, "GEM", PROCESSED_DIR / "Deutschlandatlas_GEM_merged.parquet"),
        "3": (BASE_VBGEM, "VBGEM", PROCESSED_DIR / "Deutschlandatlas_VBGEM_merged.parquet"),
    }

    if choice in mapping:
        base_dir, prefix, parquet_path = mapping[choice]
        if not parquet_path.exists():
            raise SystemExit(f"Parquet nicht gefunden: {parquet_path}")
        process_one(base_dir, prefix, indicator_labels, indicator_meta, parquet_path)
    else:
        raise SystemExit("Ungültige Auswahl.")


if __name__ == "__main__":
    main()

[labels] Lade Indikator-Labels aus c:\Users\marius.brede\Nextcloud\tmp\dev\Zeitreihenanalyse - Dash Dashboard\Version 17\geospacial\data_input\deutschlandatlas_services_mit_counts.csv
[labels] Fertig, 102 Label-Einträge.
[meta] Lade Indikator-Metadaten aus c:\Users\marius.brede\Nextcloud\tmp\dev\Zeitreihenanalyse - Dash Dashboard\Version 17\geospacial\data_input\deutschlandatlas_indikatoren.csv
[meta] Fertig, 86 Metadaten-Einträge.

Welche Ebene willst du mappen?
  1 = Kreise (KRS)
  2 = Gemeinden (GEM)
  3 = Gemeindeverband (VBGEM)
→ [KRS] erkannte Schlüsselspalte: GKZ

Verfügbare Spalten in Deutschlandatlas_KRS_merged.parquet:
  1: GKZ
  2: Kreisname
  3: fl_suv – Anteil der Siedlungs- und Verkehrsfläche an der Gesamtfläche im Jahr 2020 in %
  4: fl_landw – Anteil der Landwirtschaftsfläche an der Gesamtfläche im Jahr 2020 in %
  5: fl_wald – Anteil der Waldfläche an der Gesamtfläche im Jahr 2020 in %
  6: bev_binw – Saldo der Binnenwanderungen pro 10.000 Einwohner/-innen im Jahr 2020